In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import pandas as pd
from tqdm import tqdm  # Per la barra di caricamento

In [1]:
def test_model_alexnet(model_path, num_classes=38):
    # Percorso al CSV (puoi definirlo qui)
    csv_path = 'final_dataset/test/df_paths_test.csv'

    # Funzione per caricare il modello
    def load_model(model_path, device, num_classes):
        model = models.alexnet(pretrained=True)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)  # Modifica il classificatore finale
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        return model

    # Funzione per caricare un'immagine e applicare le trasformazioni
    def process_image(image_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)
        return image

    # Funzione per calcolare l'accuratezza
    def calculate_accuracy(predictions, targets):
        correct = (predictions == targets).sum().item()
        total = len(targets)
        accuracy = correct / total
        return accuracy

    # Carica il dispositivo (GPU se disponibile, altrimenti CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Carica il modello
    model = load_model(model_path, device, num_classes)

    # Leggi il CSV
    df = pd.read_csv(csv_path)

    # Inizializza le liste per le predizioni e i target
    predictions = []
    targets = []

    # Itera attraverso i dati di test e fai previsioni
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Testing"):
        image_path = row['FilePath']  # Assicurati che il CSV abbia una colonna 'FilePath'
        image = process_image(image_path).to(device)

        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
        
        predictions.append(predicted.item())
        targets.append(row['Classe'])  # Assicurati che il CSV abbia una colonna 'Classe' con le etichette delle classi

    # Converti le etichette delle classi a numeri se necessario
    unique_labels = df['Classe'].unique()
    label_to_index = {label: index for index, label in enumerate(unique_labels)}
    targets = [label_to_index[label] for label in targets]

    # Calcola l'accuratezza
    accuracy = calculate_accuracy(torch.tensor(predictions), torch.tensor(targets))

    # Stampa l'accuratezza
    print(f'Accuracy: {accuracy:.4f}')

# Primo test del modello AlexNet 

In [ ]:
model_path = 'models/test11_alex_net_one_hot/checkpoint_epoch_1.pt'
test_model_alexnet(model_path)

# Primo test del modello ResNet18

In [5]:
def test_model_resnet(model_path, num_classes=38):
    # Percorso al CSV (puoi definirlo qui)
    csv_path = 'final_dataset/test/df_paths_test.csv'

    # Funzione per caricare il modello
    def load_model(model_path, device, num_classes):
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modifica il classificatore finale
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        return model

    # Funzione per caricare un'immagine e applicare le trasformazioni
    def process_image(image_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)
        return image

    # Funzione per calcolare l'accuratezza
    def calculate_accuracy(predictions, targets):
        correct = (predictions == targets).sum().item()
        total = len(targets)
        accuracy = correct / total
        return accuracy

    # Carica il dispositivo (GPU se disponibile, altrimenti CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Carica il modello
    model = load_model(model_path, device, num_classes)

    # Leggi il CSV
    df = pd.read_csv(csv_path)

    # Inizializza le liste per le predizioni e i target
    predictions = []
    targets = []

    # Itera attraverso i dati di test e fai previsioni
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Testing"):
        image_path = row['FilePath']  # Assicurati che il CSV abbia una colonna 'FilePath'
        image = process_image(image_path).to(device)

        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
        
        predictions.append(predicted.item())
        targets.append(row['Classe'])  # Assicurati che il CSV abbia una colonna 'Classe' con le etichette delle classi

    # Converti le etichette delle classi a numeri se necessario
    unique_labels = df['Classe'].unique()
    label_to_index = {label: index for index, label in enumerate(unique_labels)}
    targets = [label_to_index[label] for label in targets]

    # Calcola l'accuratezza
    accuracy = calculate_accuracy(torch.tensor(predictions), torch.tensor(targets))

    # Stampa l'accuratezza
    print(f'Accuracy: {accuracy:.4f}')

In [6]:
model_path = 'models/test13_res_net_one_hot/checkpoint_epoch_1.pt'
test_model_resnet(model_path)

C:\Users\gio22\anaconda3\envs\bio\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\gio22/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:30<00:00, 3.33MB/s]
Testing: 100%|██████████| 7689/7689 [33:47<00:00,  3.79it/s]

Accuracy: 0.1812


# Secondo test del modello ResNet50

In [8]:
model_path = 'models/test15_res_net_one_hot_balanced/checkpoint_epoch_1.pt'
test_model_resnet(model_path)

C:\Users\gio22\anaconda3\envs\bio\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\gio22\anaconda3\envs\bio\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Testing: 100%|██████████| 7689/7689 [30:15<00:00,  4.24it/s] 

Accuracy: 0.1419
